In [1]:
library(feather)
library(scran)

Loading required package: BiocParallel

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colMeans,
    colnames, colSums, dirname, do.call, duplicated, eval, evalq,
    Filter, Find, get, grep, grepl, intersect, is.unsorted, lapply,
    lengths, Map, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, Positi

In [ ]:
#read in cells exp post qc and annoations
qc_cells_path = '../data/processed/qc_cells.feather'
qc_annot_path =  '../data/processed/qc_annot.csv'
qc_cells = read_feather(qc_cells_path)
qc_annot = read.csvr(qc_annot_path)

In [ ]:
#create a SingleCell experiment object
sce_qc = SingleCellExperiment(list(counts=t(as.matrix(qc_cells))))


In [ ]:
#compute size factors using the method from https://doi.org/10.1186/s13059-016-0947-7
sce_qc = computeSumFactors(sce_qc, clusters=qc_annot$pre_cluster)


In [ ]:
#normalize via sum factors
sce_qc = normalize(sce_qc)


In [ ]:
#add size factors to annoation for later analysis
qc_annot["size_factor"] = sizeFactors(sce_qc)


In [ ]:
plot(qc_annot["size_factor"], qc_annot["nUMIs"], xlab="Size Factor", ylab="Number of UMIs")


In [ ]:
plot(qc_annot["size_factor"], qc_annot["nGenes"], xlab="Size Factor", ylab="Number of genes")


In [ ]:
hist(qc_annot["size_factor"],breaks=20)


In [ ]:
#write normalized counts to file
normalized_logcounts = as.matrix(sce_qc@assays[[2]])
normalized_df = as.data.frame(t(normalized_logcounts))
colnames(normalized_df)= colnames(qc_cells)
norm_cells_path = '../processed/norm_qc_cells.feather'
write_feather(normalized_df,norm_cells_path)



In [ ]:
#write update annoation to file
